In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training2-titanic-data/titanic.csv


In [ ]:
## DAY-1
data = []
n= int(input('number of employees: '))
for i in range(n):
    print(f"\nEnter details for employee {i+1}:")
    name = input("Name: ")
    age = int(input("Age: "))
    department = input("Department: ")
    salary = int(input("Salary: "))
    
    # append to list as dictionary
    data.append({
        "name": name,
        "age": age,
        "department": department,
        "salary": salary
    })           

df = pd.DataFrame(data)

df.head(3)

df[['name', 'salary']]

avg_sal = df['salary'].mean()
avg_sal

max_sal = df['salary'].max()
print(df['name'][df['salary'] == max_sal])

np.std(df['salary'])

age = np.array(df['age'])
double = age*2
double

In [ ]:
# DAY-3
df = pd.read_csv('/kaggle/input/training2-titanic-data/titanic.csv')
df

In [ ]:
df.isnull().sum()

In [ ]:
df['Age'] = df['Age'].fillna(df['Age'].mean())

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['PassengerId', 'Cabin', 'Name', 'Ticket'], inplace=True)

In [ ]:
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

In [ ]:
df.head(10)

In [ ]:
df.isnull().sum().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode Sex
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])
df

In [ ]:
df['Embarked'].value_counts()

In [ ]:
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

In [ ]:
df

In [ ]:
df['Embarked_Q'] = df['Embarked_Q'].astype(int)
df['Embarked_S'] = df['Embarked_S'].astype(int)


In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Survived', axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Define models in a dictionary
model = RandomForestClassifier(n_estimators=200,random_state = 42)
# Loop through models
model.fit(X_train, y_train)
y_pred = model.predict(X_test)    
acc = accuracy_score(y_test, y_pred)
print(f" Accuracy: {acc:.4f}")
print(classification_report(y_test, y_pred))
print("-" * 50)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['Age', 'Fare']] = scaler.fit_transform(df[['Age', 'Fare']])
df

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42, stratify=y)

train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class titanicNN(nn.Module):
    def __init__(self, inp):
        super(titanicNN, self).__init__()
        self.fc1 = nn.Linear(inp, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64,128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

model = titanicNN(X.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 1000
for EPOCH in range(epochs):
    model.train()
    for x_batch , y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    if (EPOCH+1)%10 == 0:
        print(f"Epoch {EPOCH+1}/{epochs}, Loss: {loss.item():.4f}")

model.eval()
with torch.no_grad():
    y_pred_probs = model(X_test_t)
    y_pred_labels = (y_pred_probs > 0.5).int()
    acc = (y_pred_labels.eq(y_test_t.int()).sum().item()) / y_test_t.shape[0]
    print(f"Test Accuracy: {acc:.4f}")

In [ ]:
#### DAY-4
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report,
    confusion_matrix, roc_auc_score, roc_curve
)

def run(show_plot=True, verbose=True, random_state=69):
    data = load_breast_cancer()
    X, y = data.data, data.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state, stratify=y
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = LogisticRegression(
        max_iter=1000, solver='lbfgs', C=1.0, random_state=random_state
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    metrics = {
        'model': 'LogisticRegression',
        'accuracy': accuracy_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_pred),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred, digits=4)
    }

    if verbose:
        print('======= LOGISTIC REGRESSION =======')
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"F1 Score: {metrics['f1']:.4f}")
        print(f"AUC: {metrics['auc']:.4f}")
        print("Confusion Matrix:\n", metrics['confusion_matrix'])
        print("Classification Report:\n", metrics['classification_report'])

    if show_plot:
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure()
        plt.plot(fpr, tpr, label=f"AUC = {metrics['auc']:.4f}")
        plt.plot([0, 1], [0, 1], '--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Logistic Regression ROC Curve')
        plt.legend()
        plt.show()

    return metrics


if __name__ == '__main__':
    run()


In [ ]:
### DAY-5
### DECISION TREES

import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report,
    confusion_matrix, roc_auc_score, roc_curve
)

def run(show_plot=True, verbose=True, random_state=69):
    data = load_breast_cancer()
    X, y = data.data, data.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state, stratify=y
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = DecisionTreeClassifier(
        criterion="gini",   # or "entropy"
        max_depth=None,     # try limiting to prevent overfitting
        random_state=random_state
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    metrics = {
        'model': 'DecisionTree',
        'accuracy': accuracy_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_pred),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred, digits=4)
    }

    if verbose:
        print('======= DECISION TREE =======')
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"F1 Score: {metrics['f1']:.4f}")
        print(f"AUC: {metrics['auc']:.4f}")
        print("Confusion Matrix:\n", metrics['confusion_matrix'])
        print("Classification Report:\n", metrics['classification_report'])

    if show_plot:
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure()
        plt.plot(fpr, tpr, label=f"AUC = {metrics['auc']:.4f}")
        plt.plot([0, 1], [0, 1], '--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Decision Tree ROC Curve')
        plt.legend()
        plt.show()

    return metrics


if __name__ == '__main__':
    run()


In [ ]:
### DAY-5
### RANDOM FOREST

import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report,
    confusion_matrix, roc_auc_score, roc_curve
)

def run(show_plot=True, verbose=True, random_state=69):
    data = load_breast_cancer()
    X, y = data.data, data.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state, stratify=y
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = RandomForestClassifier(
        n_estimators=200,      # number of trees (more trees = better but slower)
        max_depth=10,          # prevent trees from growing too deep
        min_samples_split=5,   # minimum samples to split a node
        min_samples_leaf=2,    # minimum samples per leaf
        max_features="sqrt",   # best practice for classification
        bootstrap=True,        # standard RF bootstrapping
        random_state=random_state,
        n_jobs=-1              # use all CPU cores
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    metrics = {
        'model': 'RandomForest',
        'accuracy': accuracy_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_prob),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred, digits=4)
    }

    if verbose:
        print('======= RANDOM FOREST =======')
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"F1 Score: {metrics['f1']:.4f}")
        print(f"AUC: {metrics['auc']:.4f}")
        print("Confusion Matrix:\n", metrics['confusion_matrix'])
        print("Classification Report:\n", metrics['classification_report'])

    if show_plot:
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure()
        plt.plot(fpr, tpr, label=f"AUC = {metrics['auc']:.4f}")
        plt.plot([0, 1], [0, 1], '--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Random Forest ROC Curve')
        plt.legend()
        plt.show()

    return metrics


if __name__ == '__main__':
    run()


In [ ]:
### DAY-5
### SUPPORT VECTOR MACHINE (SVC)

import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report,
    confusion_matrix, roc_auc_score, roc_curve
)

def run(show_plot=True, verbose=True, random_state=69):
    data = load_breast_cancer()
    X, y = data.data, data.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state, stratify=y
    )

    # Pipeline: StandardScaler -> SVC
    model = Pipeline([
        ("scaler", StandardScaler()),
        ("svc", SVC(
            kernel="rbf",        # RBF is the default
            C=10.0,              # regularization strength (tune this)
            gamma="scale",       # kernel coefficient
            probability=True,    # enable predict_proba
            random_state=random_state
        ))
    ])

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    metrics = {
        'model': 'SVC',
        'accuracy': accuracy_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_prob),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred, digits=4)
    }

    if verbose:
        print('======= SUPPORT VECTOR MACHINE =======')
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"F1 Score: {metrics['f1']:.4f}")
        print(f"AUC: {metrics['auc']:.4f}")
        print("Confusion Matrix:\n", metrics['confusion_matrix'])
        print("Classification Report:\n", metrics['classification_report'])

    if show_plot:
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure()
        plt.plot(fpr, tpr, label=f"AUC = {metrics['auc']:.4f}")
        plt.plot([0, 1], [0, 1], '--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('SVM ROC Curve')
        plt.legend()
        plt.show()

    return metrics


if __name__ == '__main__':
    run()

In [ ]:
### DAY 6
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(8 * 13 * 13, 10)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(-1, 8 * 13 * 13)
        x = self.fc1(x)
        return x

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(
    root='/kaggle/working',
    train=True,
    download=True,
    transform=transform
)
testset = torchvision.datasets.MNIST(
    root='/kaggle/working',
    train=False,
    download=True,
    transform=transform
)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

print("Train batches:", len(trainloader))
print("Test batches:", len(testloader))


In [ ]:
### DAY 8
### REGULARIZED CNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
class regCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)   
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)               
        self.fc1 = nn.Linear(16 * 14 * 14, 10)

    def forward(self, x):
        x = self.pool(self.bn1(torch.relu(self.conv1(x))))
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        return self.fc1(x)
        
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)
testset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = regCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
EPOCHS = 5
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {running_loss/len(trainloader):.4f}")

print("Training done")


In [8]:
### DAY- 13

# word2vec

import torch 
import torch.nn as nn
import torch.optim as optim

corpus= 'king queen man woman'
words= corpus.split()
vocab= list(set(words))
word_to_idx= {word: i for i, word in enumerate(vocab)}
idx_to_word= {i: word for word,i in word_to_idx.items()}
pairs= [(words[i], words[i+1]) for i in range(len(words)-1)]

class word2vec(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.emb= nn.Embedding(vocab_size, embed_dim)
        self.linear= nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        return self.linear(self.emb(x))

model = word2vec(len(vocab), 5)
loss_fn= nn.CrossEntropyLoss()
opt= optim.SGD(model.parameters(), lr= 1e-5)
for epoch in range(50):
    total_loss= 0
    for center, context in pairs:
        context_idx= torch.tensor([word_to_idx[context]])
        center_idx= torch.tensor([word_to_idx[center]])

        opt.zero_grad()
        pred= model(center_idx)
        loss= loss_fn(pred, context_idx)
        loss.backward()
        opt.step()
        total_loss+= loss.item()
    print(f'EPOCH {epoch}, loss: {total_loss/len(pairs)}')

embeddings= model.emb.weight.data
print('vector for king:', embeddings[word_to_idx['king']])
print('vector for queen:', embeddings[word_to_idx['queen']])

EPOCH 0, loss: 1.6513217290242512
EPOCH 1, loss: 1.6512994368871052
EPOCH 2, loss: 1.651277224222819
EPOCH 3, loss: 1.6512549718221028
EPOCH 4, loss: 1.6512326796849568
EPOCH 5, loss: 1.6512104272842407
EPOCH 6, loss: 1.6511882940928142
EPOCH 7, loss: 1.6511659622192383
EPOCH 8, loss: 1.6511437892913818
EPOCH 9, loss: 1.6511214971542358
EPOCH 10, loss: 1.6510993242263794
EPOCH 11, loss: 1.6510769923528035
EPOCH 12, loss: 1.6510547399520874
EPOCH 13, loss: 1.6510326067606609
EPOCH 14, loss: 1.651010274887085
EPOCH 15, loss: 1.6509881019592285
EPOCH 16, loss: 1.6509658495585124
EPOCH 17, loss: 1.650943636894226
EPOCH 18, loss: 1.6509214242299397
EPOCH 19, loss: 1.650899092356364
EPOCH 20, loss: 1.6508769989013672
EPOCH 21, loss: 1.6508547067642212
EPOCH 22, loss: 1.6508324543635051
EPOCH 23, loss: 1.6508102019627888
EPOCH 24, loss: 1.6507879495620728
EPOCH 25, loss: 1.6507657766342163
EPOCH 26, loss: 1.6507436037063599
EPOCH 27, loss: 1.650721271832784
EPOCH 28, loss: 1.6506991386413574
